![](images/vaccine-6116391_1920_pixabay.jpg)

<center><font size="7"><div style='background:green;color:white'>COVID-19 Vaccination Progress EDA</div></font></center>

***
***

<font size="6"><div style='color:green'>Table of Contents</div></font>

1. INTRODUCTION

    *1.1 COVID-19 pandemic*
    
    *1.2 COVID-19 vaccines*
    
    *1.3 Key objective*
    
    *1.4 Dataset overview*

2. IMPORTS AND DATA PREPARATION

    *2.1 Importing libraries and data*

    *2.2 Preliminary data exploration and subsequent cleaning*

    *2.3 Feature engineering to prepare for in-depth data analysis*

3. IN-DEPTH DATA ANALYSIS AND VISUALISATION

    *3.1 Distributions of .... (abs and %)*
    
    *3.2 Time-course of...*
    
    *3.3 TIme-series analysis of...*

***
***

<font size="5"><div style='background:green;color:white'><b>1. Introduction to COVID-19 Vaccines</b></div></font>

-Vaccinaction: Version 81

QUESTIONS to investigate

-What vaccines are used and in which countries?

-What country is vaccinated more people?

-What country is vaccinated a larger percent from its population?

<font size="4"><div style='background:yellow;color:black'><b>1.4 Dataset Overview</b></div></font>

**The dataset contains 8079 observations for 15 features.**

| FEATURE | description | comment |
|----|----|----|
| **country** | 151 countries | some duplicates, see UK |
| **iso_code** | country code by ISO norm | -|
| **date** | data entry (13th December 2020 to 24th March 2021) | some days only daily vaccinations, some days only the (cummulative) total |
| **vaccines** | name/company | some days only daily vaccinations, some days only the (cummulative) total|
| **daily_vaccinations** | no. of vaccination for that date/country | - |
| **daily_vaccinations_raw** | daily vaccinations as CHANGE COMPARED TO PREVIOUS DAY (sanity check) | - |
| **daily_vaccinations_per_million** | ratio (in ppm) between vaccination number and total population for the current date in the country | - |
| **total_vaccinations** | absolute number of total immunizations in the country | - |
| **total_vaccinations_per_hundred** | ratio (in percent) between vaccination number and total population up to the date in the country | - |
| **people_vaccinated** | total no. of people with immunisation (partial and full) | - |
| **people_vaccinated_per_hundred** | ratio (in percent) between population immunized (partial and full) and total population | - |
| **people_fully_vaccinated** | total no. of people with full immunisation (typically 2) | - |
| **people_fully_vaccinated_per_hundred** | ratio (in percent) between population fully immunized and total population | - |
| **source_name** | national authority, international organization, local organization etc. | - |
| **source_website** | website link | - |

In [15]:
#import basic libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
from datetime import datetime
#from pandas_profiling import ProfileReport

In [4]:
#import dataset country_vaccinations:
df = pd.read_csv('data/country_vaccinations.csv')

In [13]:
#first check:
df.head(3)

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...


In [14]:
#Number of columns and rows:
shape1 = df.shape
print(f'We have {shape1[1]} columns and {shape1[0]} rows in our dataset.')

We have 15 columns and 8079 rows in our dataset.


In [ ]:
#Information about datatypes per column:
df.info()

Cleaning task: 
- change object to datetime for column date
- Check missing values in all columns with rows less than 8079

In [ ]:
#Basic profile
#profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
#profile

Data cleaning tasks:
- countries have different names (e.g. England, United Kingdom, Northern Ireland, Wales, Scotland)
- drop daily_vaccinations_raw

In [ ]:
#copy dataset before cleaning
df1 = df.copy()

In [ ]:
#change type for date to datetime
df1['date'] =  pd.to_datetime(df1['date'])

In [ ]:
#test changes to date
df1.info()

In [ ]:
#check for duplicates in country and date:
df1[df1.duplicated(subset=['country','date'], keep=False)].shape

In [ ]:
#drop column daily_vaccinations_raw, iso_code, source_name, source_website:
lst = ['iso_code', 'daily_vaccinations_raw', 'source_name', 'source_website']
df1.drop(lst, 1, inplace=True)
df1.head(3)

In [ ]:
lst2 = ['England', 'Scotland', 'Wales', 'Northern Ireland']
check1 = df1.loc[df['country'].isin(lst2)].loc[df1['date'] == '2021-03-15'].loc[:, 'daily_vaccinations'].sum()
check2 = df1.loc[df['country'] == 'United Kingdom'].loc[df1['date'] == '2021-03-15'].loc[:, 'daily_vaccinations'].values[0]
print(f'The sum of daily vaccinations in England, Scotland, Wales and Nothern Ireland on 15th March 2021 was: {check1}')
print(f'The total daily vaccinations in the United Kingdom on 15th March 2021 was: {check2}')

In [ ]:
df1 = df1[df1.country.apply(lambda x: x not in ['England', 'Scotland', 'Wales', 'Northern Ireland'])]

In [ ]:
df.isnull().sum(axis = 0)

In [ ]:
df1['vaccines'].unique()

In [ ]:
df2 = df1.drop('vaccines', axis=1).join(df['vaccines'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).rename('vaccines'))
df3 = df2[['country', 'date', 'vaccines']]
df2.head()

In [ ]:
df4 = df3.pivot(index=['country', 'date'], columns='vaccines', values='vaccines').reset_index()
df4.sample(20)

In [ ]:
lst_ = ['country', 'date', 'vaccines']
df1.drop(lst_, 1, inplace=True)
df1.head(3)

In [ ]:
df5 = df4.join(df1)
df5.head(3)

In [ ]:
#check correlation of data
sns.pairplot(data=df5)

In [ ]:
#Which country uses which vaccines?

***
# COVID-19 Vaccination Twitter tweets
****

# Introduction

-Vaccinaction: Version 81

-Tweets: Version 43

##Track the progress of COVID-19 vaccination

Was muss für ordentliches Mark up gemacht werden?

-What vaccines are used and in which countries?

-What country is vaccinated more people?

-What country is vaccinated a larger percent from its population?

The submission evaluation will be based on the:

Originality of your solution;
Quality of data analysis;
Methods and techniques presentation;
Quality of results;
Comments and conclusions.

##Tweets
### Perform sentiment analysis on the tweets looking to things like:

- Sentiment polarity (Negative/Positive/Neutral) in relationship with each vaccine (looking to the association with the vaccine name, as found in tweets);
Evolution of sentiment in time;

Expected:
Data analysis;
Method for sentiment analysis;
Results;
Comments and conclusions.

Column Description:
id
user_name
user_location
user_description
user_created - creation of user in twitter
user_followers
user_friends
user_favourites
user_verified
date
text
hashtags
source
retweets
favorites
is_retweet

In [ ]:
#import dataset vaccination_all_tweets:
df_tweet = pd.read_csv('data/vaccination_all_tweets.csv')

In [ ]:
#first check of tweet data:
df_tweet.head()

In [ ]:
#Number of columns and rows:
shapet1 = df_tweet.shape
print(f'We have {shapet1[1]} columns and {shapet1[0]} rows in our twitter tweet dataset.')

In [ ]:
list(df_tweet)

In [ ]:
#information of data types per column:
df_tweet.info()

- change date to datetime
- check columns with rows less than 38547 for missing values

In [ ]:
profile = ProfileReport(df_tweet, title="Pandas Profiling Report", explorative=True)
profile

In [ ]:
#copy dataset before cleaning
df_tweet1 = df_tweet.copy()